In [4]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf

def load_image_dataset(base_dir, img_size=(224, 224)):
    """
    Load images from subdirectories into a single dataset.
    
    Parameters:
    base_dir (str): Path to the base directory containing class subdirectories
    img_size (tuple): Target size for the images (default: (224, 224))
    
    Returns:
    tuple: (X_data, y_data, label_map)
    """
    
    # Lists to store images and labels
    images = []
    labels = []
    
    # Create a mapping of directory names to numeric labels
    label_map = {}
    for idx, class_name in enumerate(sorted(os.listdir(base_dir))):
        label_map[class_name] = idx
    
    # Load images and labels
    for class_name in sorted(os.listdir(base_dir)):
        class_dir = os.path.join(base_dir, class_name)
        if not os.path.isdir(class_dir):
            continue
            
        class_label = label_map[class_name]
        
        for img_name in os.listdir(class_dir):
            if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                img_path = os.path.join(class_dir, img_name)
                try:
                    # Load and preprocess image
                    img = Image.open(img_path)
                    img = img.convert('RGB')  # Convert to RGB format
                    img = img.resize(img_size)  # Resize image
                    img_array = np.array(img) / 255.0  # Normalize pixel values
                    
                    images.append(img_array)
                    labels.append(class_label)
                except Exception as e:
                    print(f"Error loading image {img_path}: {str(e)}")
    
    # Convert lists to numpy arrays
    X_data = np.array(images)
    y_data = np.array(labels)
    
    return X_data, y_data, label_map


In [5]:
# Example usage
if __name__ == "__main__":
    # Replace with your directory path
    base_directory = "/kaggle/input/traffic-data/traffic_Data/DATA"
    
    # Load the dataset
    X_data, y_data, label_map = load_image_dataset(
        base_directory,
        img_size=(224, 224)
    )
    
    # Print dataset information
    print(f"Number of samples: {len(X_data)}")
    print(f"Number of classes: {len(label_map)}")
    print(f"Class mapping: {label_map}")
    
    #Convert to TensorFlow dataset
    dataset = tf.data.Dataset.from_tensor_slices((X_data, y_data))

Number of samples: 4170
Number of classes: 58
Class mapping: {'0': 0, '1': 1, '10': 2, '11': 3, '12': 4, '13': 5, '14': 6, '15': 7, '16': 8, '17': 9, '18': 10, '19': 11, '2': 12, '20': 13, '21': 14, '22': 15, '23': 16, '24': 17, '25': 18, '26': 19, '27': 20, '28': 21, '29': 22, '3': 23, '30': 24, '31': 25, '32': 26, '33': 27, '34': 28, '35': 29, '36': 30, '37': 31, '38': 32, '39': 33, '4': 34, '40': 35, '41': 36, '42': 37, '43': 38, '44': 39, '45': 40, '46': 41, '47': 42, '48': 43, '49': 44, '5': 45, '50': 46, '51': 47, '52': 48, '53': 49, '54': 50, '55': 51, '56': 52, '57': 53, '6': 54, '7': 55, '8': 56, '9': 57}


I0000 00:00:1739742912.642631      10 service.cc:148] XLA service 0x5741a0b2f040 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1739742912.642690      10 service.cc:156]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1739742912.642694      10 service.cc:156]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1739742912.642697      10 service.cc:156]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1739742912.642700      10 service.cc:156]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1739742912.642703      10 service.cc:156]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1739742912.642705      10 service.cc:156]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1739742912.642709      10 service.cc:156]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1739742912.642711      10 service.cc:156]   StreamExecutor device (7): TPU, 2a886c8


In [6]:
import cv2

def extract_edge_features(X_data):
    features_1 = []

    for img_array in X_data:
        # Convert the image array to uint8 before processing
        img = (np.array(img_array) * 255).astype(np.uint8)
        
        # Convert to grayscale using PIL (more efficient)
        gray_image = Image.fromarray(img).convert("L")

        # Convert grayscale image back to NumPy array for OpenCV processing
        gray_array = np.array(gray_image)

        # Apply the Sobel operator in the X direction
        sobel_x = cv2.Sobel(gray_array, cv2.CV_64F, 1, 0, ksize=3)

        # Apply the Sobel operator in the Y direction
        sobel_y = cv2.Sobel(gray_array, cv2.CV_64F, 0, 1, ksize=3)

        # Calculate the magnitude of the gradients
        magnitude = np.sqrt(sobel_x**2 + sobel_y**2)

        # Normalize to 0-255, avoid division by zero
        max_magnitude = np.max(magnitude)
        if max_magnitude > 0:
            magnitude = np.uint8(255 * magnitude / max_magnitude)
        else:
            magnitude = np.uint8(magnitude)  # If max is 0, no edges exist

        # Store the processed image in features_1
        features_1.append(magnitude)

    return np.array(features_1)  

In [7]:
def extract_line_features(X_data):
    features_2 = []

    for img_array in X_data:
        # Convert the image array to uint8 before processing
        img = (np.array(img_array) * 255).astype(np.uint8)
        
        # Convert to grayscale using PIL
        gray_image = Image.fromarray(img).convert("L")

        # Convert grayscale image back to NumPy array for OpenCV processing
        gray_array = np.array(gray_image)

        # Find edges using Canny edge detector
        edges = cv2.Canny(gray_array, 50, 200)

        # Detect lines using Hough Line Transform
        lines = cv2.HoughLinesP(edges, 1, np.pi/180, 68, minLineLength=15, maxLineGap=250)

        # Convert grayscale image to BGR (for colored line drawing)
        line_image = cv2.cvtColor(gray_array, cv2.COLOR_GRAY2BGR)

        # Draw detected lines on the image
        if lines is not None:
            for line in lines:
                x1, y1, x2, y2 = line[0]
                cv2.line(line_image, (x1, y1), (x2, y2), (255, 0, 0), 3)  # Blue lines

        # Store the processed image in features_2
        features_2.append(line_image)

    return np.array(features_2) 

In [8]:
def extract_contour_features(X_data):
    features_3 = []

    for img_array in X_data:
        # Convert the image array to uint8 before processing
        img = (np.array(img_array) * 255).astype(np.uint8)
        
        # Convert to grayscale using PIL
        gray_image = Image.fromarray(img).convert("L")

        # Convert grayscale image back to NumPy array for OpenCV processing
        gray_array = np.array(gray_image)

        # STEP 1: Apply GaussianBlur to reduce noise
        blurred = cv2.GaussianBlur(gray_array, (5, 5), 0)

        # STEP 2: Use Canny edge detector
        edges = cv2.Canny(blurred, 50, 150)

        # STEP 3: Find contours
        contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        # Create a color copy of the image for contour drawing (in BGR format)
        contour_image = cv2.cvtColor(gray_array, cv2.COLOR_GRAY2BGR)

        # Draw contours in green (0, 255, 0) with thickness of 1
        cv2.drawContours(contour_image, contours, -1, (0, 255, 0), 1)

        # Store the processed contour image in features_3
        features_3.append(contour_image)

    return np.array(features_3)  

In [9]:
features_1 = extract_edge_features(X_data)
features_2 = extract_line_features(X_data)
features_3 = extract_contour_features(X_data)

In [10]:
if features_2.shape[-1] == 3:  # Check if last dimension is 3 (RGB)
    features_2_new = np.array([cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) for img in features_2])
else:
    print("Already grayscale, skipping conversion!")
    features_2_new = features_2  # No need to convert

print(features_2_new.shape)  # Should be (4170, 224, 224)


(4170, 224, 224)


In [11]:
if features_3.shape[-1] == 3:  # Check if last dimension is 3 (RGB)
    features_3_new = np.array([cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) for img in features_3])
else:
    print("Already grayscale, skipping conversion!")
    features_3_new = features_3  # No need to convert

print(features_3_new.shape)  # Should be (4170, 224, 224)


(4170, 224, 224)


In [12]:
import numpy as np

def preprocess_and_concatenate_features(features_1,features_2, features_3):
    
    # Get the number of samples
    n_samples = features_3.shape[0]
    
    # Reshape 3D feature (4170, 224, 224) -> (4170, 224*224)
    feature_1_reshaped = features_3.reshape(n_samples, -1)
    
    # Reshape 4D feature (4170, 224, 224, 3) -> (4170, 224*224*3)
    feature_2_reshaped = features_2.reshape(n_samples, -1)

    feature_3_reshaped = features_3.reshape(n_samples, -1)
    
    # Concatenate along feature axis
    concatenated_features = np.concatenate(
        (feature_1_reshaped, feature_2_reshaped,feature_3_reshaped), 
        axis=1
    )
    
    print(f"Original 3D feature shape: {features_1.shape}")
    print(f"Original 4D feature shape: {features_2.shape}")
    print(f"Original 4D feature shape: {features_3.shape}")
    print(f"Reshaped 3D feature shape: {feature_1_reshaped.shape}")
    print(f"Reshaped 4D feature shape: {feature_2_reshaped.shape}")
    print(f"Reshaped 4D feature shape: {feature_3_reshaped.shape}")
    print(f"Final concatenated shape: {concatenated_features.shape}")
    
    return concatenated_features

In [13]:
concatenated = preprocess_and_concatenate_features(features_1, features_2_new, features_3_new)
np.shape(concatenated)

Original 3D feature shape: (4170, 224, 224)
Original 4D feature shape: (4170, 224, 224)
Original 4D feature shape: (4170, 224, 224)
Reshaped 3D feature shape: (4170, 50176)
Reshaped 4D feature shape: (4170, 50176)
Reshaped 4D feature shape: (4170, 50176)
Final concatenated shape: (4170, 150528)


(4170, 150528)

In [14]:
from sklearn.decomposition import PCA

pca = PCA(n_components=500)  # Reduce to 500 dimensions
X_pca = pca.fit_transform(concatenated)
np.shape(X_pca)

(4170, 500)

In [15]:
from sklearn.model_selection import train_test_split

X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(X_pca, y_data, test_size=0.2, random_state=42)


In [16]:
from sklearn.svm import SVC

svm_rbf = SVC(kernel='rbf', C=1.0, gamma='scale', cache_size=7000)  # Increase cache size
svm_rbf.fit(X_train_new, y_train_new)


SVC(cache_size=7000)

In [17]:
#with dimensionality reduction 
accuracy = svm_rbf.score(X_test_new, y_test_new)
print(f"Test Accuracy: {accuracy:.2f}")

Test Accuracy: 0.88
